In [14]:
import tensorflow as tf
import keras
import numpy as np

from keras import Input, layers
from keras.models import Model, load_model
from keras.layers import Input, Add, Dense, Activation, BatchNormalization
from keras.layers import ZeroPadding1D, Conv1D, MaxPooling1D, AveragePooling1D, GlobalMaxPooling1D
from keras.layers import Dropout, LSTM, Bidirectional, concatenate

from keras.utils import to_categorical

print(tf.__version__)
print(keras.__version__)

2.1.0
2.3.1


### Building Blocks

In [2]:
# Identity Block - IITNet
def identity_block_iitnet(input_x, kernel_size, filters, stage, block, branch):
    """
    Implementation of the identity block (or bottleneck block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras. Altered as described 
    in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        branch: index of the input branch
        block: 'a','b'..., current block label, used for generating layer names
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'branch' + branch + '_res' + str(stage) + block + '_branch'
    bn_name_base = 'branch' + branch + '_bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv1D(filters = filters1,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv1D(filters = filters2,
               kernel_size = kernel_size,
               strides = 1,
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv1D(filters = filters3,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(name = bn_name_base + '2c')(x)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [3]:
# Convolutional Block
def convolutional_block_iitnet(input_x, kernel_size, filters, stage, block, branch, strides=(2, 2)):
    """
    Implementation of the convolutional (or standard block) as described in 
    https://arxiv.org/pdf/1512.03385.pdf and implemented in Keras. Altered as described 
    in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        kernel_size: default 3, the kernel size of middle conv layer for the main path
        filters: list of integers, the number of filters in the conv layers at the main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        branch: index of the input branch
        stride: strides for the first convolutional layer in the block
    
    # Returns
        Output tensor
    """
    
    #define name basis for the layers in the block
    conv_name_base = 'branch' + branch + '_res' + str(stage) + block + '_branch'
    bn_name_base = 'branch' + branch + '_bn' + str(stage) + block + '_branch'
    
    #get the filter numbers for the three conv layers
    filters1, filters2, filters3 = filters
    
    #store the input tensor for later
    x_shortcut = input_x
    
    #first component (main path)
    x = Conv1D(filters = filters1,
               kernel_size = 1,
               strides = strides,
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2a')(input_x)
    x = BatchNormalization(name = bn_name_base + '2a')(x)
    x = Activation('relu')(x)
    
    #second component (main path)
    x = Conv1D(filters = filters2,
               kernel_size = kernel_size,
               strides = 1,
               padding = 'same',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2b')(x)
    x = BatchNormalization(name = bn_name_base + '2b')(x)
    x = Activation('relu')(x)
    
    #third component (main path)
    x = Conv1D(filters = filters3,
               kernel_size = 1,
               strides = 1,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = conv_name_base + '2c')(x)
    x = BatchNormalization(name = bn_name_base + '2c')(x)
    
    #shortcut path
    x_shortcut = Conv1D(filters = filters3,
                        kernel_size = 1,
                        strides = strides,
                        kernel_initializer = 'he_normal',
                        name = conv_name_base + '1')(x_shortcut)
    x_shortcut = BatchNormalization(name = bn_name_base + '1')(x_shortcut)
    
    #final step: addition of main path and recurrent shortcut, activation
    x = layers.add([x, x_shortcut])
    x = layers.Activation('relu')(x)
    
    return x

In [4]:
def ResNet50(x_input, branch=1):
    """
    Build the ResNet50 architecture as described in https://arxiv.org/pdf/1512.03385.pdf 
    and implemented in Keras. Altered as described in Back (2019)
    
    # Arguments
        input_x: Input tensor 
        branch: index of the input branch
        
    # Returns
        Output tensor
    """
    
    branch_index = str(branch)
    
    #Zero Padding
    x = ZeroPadding1D(padding = 3, name = 'conv1_pad' + branch_index)(x_input)
    
    #Stage 1
    x = Conv1D(8, 2, #resnet: 64, 7, 
               strides = 2,
               padding = 'valid',
               kernel_initializer = 'he_normal',
               name = 'conv1' + branch_index)(x)
    x = BatchNormalization(name = 'bn_conv1' + branch_index)(x)
    x = Activation('relu')(x)
    x = ZeroPadding1D(padding = 1, 
                      name = 'pool1_pad' + branch_index)(x)
    x = MaxPooling1D( 3, 
                     strides = 2)(x)
    
    print(x)
    
    #Stage 2
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [8, 8, 16], #resnet: [64, 64, 256]
                            stage = 2, 
                            block = 'a',
                            branch = branch_index,
                            strides = 1)
    x = identity_block_iitnet(input_x = x,
                                kernel_size = 3,
                                filters = [8, 8, 16], #resnet: [64, 64, 256]
                               stage = 2,
                               block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [8, 8, 16], #resnet: [64, 64, 256]
                       stage = 2,
                       block = 'c',
                        branch = branch_index)
    
    print(x)
    
    #Stage 3
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [16, 16, 32], #resnet: [128, 128, 512]
                            stage = 3, 
                            block = 'a', 
                            strides = 2,
                            branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'b',
                        branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'c',
                        branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [16, 16, 32], #resnet: [128, 128, 512]
                       stage = 3,
                       block = 'd',
                        branch = branch_index)
    
    print(x)
    
    #Additional max-pool to reduce the length of the feature sequence by half
    x = MaxPooling1D()(x)
    
    print(x)
    
    #Stage 4
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [32, 32, 64], #resnet: [256, 256, 1024] 
                            stage = 4, 
                            block = 'a', 
                            strides = 2,
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'c',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'd',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'e',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [32, 32, 64], #resnet: [256, 256, 1024]
                       stage = 4,
                       block = 'f',
                              branch = branch_index)
    
    print(x)
    
    #Stage 5
    x = convolutional_block_iitnet(input_x = x, 
                            kernel_size = 3, 
                            filters = [64, 64, 128], #resnet: [512, 512, 2048]
                            stage = 5, 
                            block = 'a', 
                            strides = 2,
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 128], #resnet: [512, 512, 2048]
                       stage = 5,
                       block = 'b',
                              branch = branch_index)
    x = identity_block_iitnet(input_x = x,
                       kernel_size = 3,
                       filters = [64, 64, 128], #resnet: [512, 512, 2048]
                       stage = 5,
                       block = 'c',
                              branch = branch_index)
    
    print(x)
    
    #dropout layer to prevent overfitting
    x = Dropout(0.5)(x)
    
    print(x)
    
    return x

### Simplified IITNet

In [5]:
#build a simplified IITNet, with the number of epochs and subepochs = 1
def simple_iitnet(n_epochs=1, n_subepochs=1, n_channels=1, frequency=200, classes=5):
    """
    Build the IITNet architecture as described in Back (2019). This is a simplified version, which only
    works with a single 30-second sequence (epoch) of a single-channel EEG. This epoch does not get 
    divided in subepochs (number of subepochs = 1).
    
    # Arguments
        n_epochs = number of time epochs to be used for classification
        n_subepochs = number of subparts every epoch is to be split into
        n_channels = used number of EEG/EOG/... channels
        frequency = number of measured values per second
        classes = number of classes (sleepphases) to classify epochs into
        
    # Returns
        classification model
    """
    
    #define input to the model as a tensor
    subepoch_length = 30 / n_subepochs
    n_datapoints = subepoch_length * frequency
    
    epochs_total = n_epochs * n_subepochs
    
    input_shape = (n_channels, int(n_datapoints))
    x_input = Input(input_shape)
    
    x = keras.layers.Reshape((int(n_datapoints), n_channels),
                                    input_shape=(input_shape))(x_input)
    
    print(x)
    
    #convolutional layers
    x = ResNet50(x)
    
    #recurrent layers
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    x = Bidirectional(LSTM(128))(x)
    
    #fully connected layer + classifier
    x = Dense(classes,
              activation = 'softmax',
              name = 'fc' + str(classes))(x)    
    
    #create model
    model = Model(x_input, x, name = 'Simple_IITNet')
    
    return model

In [8]:
model_simple_iitnet = simple_iitnet()
model_simple_iitnet.summary()

Tensor("reshape_2/Reshape:0", shape=(None, 6000, 1), dtype=float32)
Tensor("max_pooling1d_2/Squeeze:0", shape=(None, 1502, 8), dtype=float32)
Tensor("activation_13/Relu:0", shape=(None, 1502, 16), dtype=float32)
Tensor("activation_25/Relu:0", shape=(None, 751, 32), dtype=float32)
Tensor("max_pooling1d_3/Squeeze:0", shape=(None, 375, 32), dtype=float32)
Tensor("activation_43/Relu:0", shape=(None, 188, 64), dtype=float32)
Tensor("activation_52/Relu:0", shape=(None, 94, 128), dtype=float32)
Tensor("dropout_1/cond/Identity:0", shape=(None, 94, 128), dtype=float32)
Model: "Simple_IITNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 6000)      0                                            
__________________________________________________________________________________________________
reshape_2 (Res

## Get Data

In [10]:
#preprocessed data
file_path_x = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/physionet_challenge/577000099446A/tr03-0005/0_data_x_preprocessed.npy"
file_path_y = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/physionet_challenge/577000099446A/tr03-0005/0_data_y_preprocessed.npy"

In [11]:
data_x = np.load(file_path_x)
data_y = np.load(file_path_y)

print(data_x.shape)
print(data_y.shape)

(857, 6000, 13)
(857, 1)


In [12]:
data_x.shape[0]

857

In [13]:
print(data_x)
print(data_y)

[[[   -9     5    -5 ...    39 30496   -53]
  [  -36     2   -33 ...    93 30496  -103]
  [  -23     2   -23 ...    62 30496     6]
  ...
  [   17    23    19 ...    18 30496   -90]
  [   21    19    24 ...    -2 30496    20]
  [   24    12    26 ...   -22 30496    91]]

 [[    4     4     8 ...    10 30496   -67]
  [    5     1     9 ...    11 30496   -44]
  [   19     7    22 ...   -22 30496   102]
  ...
  [   72     7    70 ...    11 30496   -15]
  [   86     2    84 ...   -19 30496   112]
  [   72     0    71 ...     9 30496   -49]]

 [[   66     4    68 ...    21 30496   -76]
  [   78     7    79 ...   -14 30496   102]
  [   68     6    69 ...    -5 30496    20]
  ...
  [  -44   -11   -45 ...   -62 30496  -463]
  [  -60   -10   -61 ...   -32 30496  -547]
  [  -46    -8   -46 ...   -52 30496  -460]]

 ...

 [[  -26   -10   -19 ...    26 31145 -1094]
  [  -23    -9   -17 ...    24 31145   -39]
  [  -19    -5   -14 ...    11 31145   954]
  ...
  [  -11    -7     2 ...     0 30496   7

## Training Data

### Datapoints (Single EEG channel)

In [295]:
data_x = data_x[:,:,0]
print(data_x.shape)
print(data_x)

(857, 6000)
[[ -9 -36 -23 ...  17  21  24]
 [  4   5  19 ...  72  86  72]
 [ 66  78  68 ... -44 -60 -46]
 ...
 [-26 -23 -19 ... -11  -9  -6]
 [ -6 -12 -18 ...  -5  -2   0]
 [  2   4   5 ...   2  -2  -3]]


In [296]:
data_x = data_x.reshape((857,1,6000))
print(data_x)

[[[ -9 -36 -23 ...  17  21  24]]

 [[  4   5  19 ...  72  86  72]]

 [[ 66  78  68 ... -44 -60 -46]]

 ...

 [[-26 -23 -19 ... -11  -9  -6]]

 [[ -6 -12 -18 ...  -5  -2   0]]

 [[  2   4   5 ...   2  -2  -3]]]


### Datapoints (Two EEG channels)

In [41]:
data_x = data_x[:,:,:2]
print(data_x.shape)
print(data_x)

(857, 6000, 2)
[[[ -9   5]
  [-36   2]
  [-23   2]
  ...
  [ 17  23]
  [ 21  19]
  [ 24  12]]

 [[  4   4]
  [  5   1]
  [ 19   7]
  ...
  [ 72   7]
  [ 86   2]
  [ 72   0]]

 [[ 66   4]
  [ 78   7]
  [ 68   6]
  ...
  [-44 -11]
  [-60 -10]
  [-46  -8]]

 ...

 [[-26 -10]
  [-23  -9]
  [-19  -5]
  ...
  [-11  -7]
  [ -9  -3]
  [ -6  -1]]

 [[ -6  -2]
  [-12  -5]
  [-18  -5]
  ...
  [ -5  -5]
  [ -2  -7]
  [  0  -8]]

 [[  2  -7]
  [  4  -5]
  [  5  -3]
  ...
  [  2  -8]
  [ -2  -6]
  [ -3  -4]]]


In [42]:
number_epochs = data_x.shape[0]
data_x = data_x.reshape((number_epochs,2,6000))
print(data_x)

[[[  -9    5  -36 ...  -26  -35  -25]
  [ -21  -19  -14 ...   19   24   12]]

 [[   4    4    5 ...    4 -275    1]
  [-269   -2 -289 ...    2   72    0]]

 [[  66    4   78 ...  -14   95  -13]
  [  82  -18   79 ...  -10  -46   -8]]

 ...

 [[ -26  -10  -23 ...    8    0    8]
  [   2    8    4 ...   -3   -6   -1]]

 [[  -6   -2  -12 ...   -4   -8   -1]
  [  -6    1   -6 ...   -7    0   -8]]

 [[   2   -7    4 ...    6   32    5]
  [  27    5   27 ...   -6   -3   -4]]]


### Labels

In [15]:
data_y = to_categorical(data_y)
print(data_y.shape)
print(data_y[:5])

(857, 5)
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]


## Test Data

In [45]:
#test data
test_path_x = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/physionet_challenge/577000099446A/tr03-0029/0_data_x_preprocessed.npy"
test_path_y = "C:/Users/sa6pr7/PycharmProjects/medical-icu/src/data/physionet_challenge/577000099446A/tr03-0029/0_data_y_preprocessed.npy"

### Test Data (single channel)

In [33]:
test_data_x = np.load(file_path_x)
test_data_y = np.load(file_path_y)

test_data_x = test_data_x[:,:,0]
test_data_y = test_data_x.reshape((857,1,6000))

test_data_y = to_categorical(test_data_y)
test_data_y = test_data_y[:,:,1]
#test_data_y = data_y.reshape((857,5))

### Test Data (two EEG channels)

In [50]:
test_data_x = np.load(test_path_x)
test_data_y = np.load(test_path_y)


test_data_x = test_data_x[:,:,:2]
number_epochs = test_data_x.shape[0]
test_data_x = test_data_x.reshape((number_epochs,2,6000))

test_data_y = to_categorical(test_data_y)
#test_data_y = data_y[:]
#test_data_y = data_y.reshape((857,5))

# Model Training

In [16]:
#single channel
#model_simple_iitnet = simple_iitnet(n_channels=1)

#two channels
model_simple_iitnet = simple_iitnet(n_channels=2)

model_simple_iitnet.summary()

Tensor("reshape_3/Reshape:0", shape=(None, 6000, 2), dtype=float32)
Tensor("max_pooling1d_4/Squeeze:0", shape=(None, 1502, 8), dtype=float32)
Tensor("activation_62/Relu:0", shape=(None, 1502, 16), dtype=float32)
Tensor("activation_74/Relu:0", shape=(None, 751, 32), dtype=float32)
Tensor("max_pooling1d_5/Squeeze:0", shape=(None, 375, 32), dtype=float32)
Tensor("activation_92/Relu:0", shape=(None, 188, 64), dtype=float32)
Tensor("activation_101/Relu:0", shape=(None, 94, 128), dtype=float32)
Tensor("dropout_2/cond/Identity:0", shape=(None, 94, 128), dtype=float32)
Model: "Simple_IITNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 2, 6000)      0                                            
__________________________________________________________________________________________________
reshape_3 (Re

In [17]:
model_simple_iitnet.compile(optimizer='adam',
                            loss='categorical_crossentropy',
                            metrics=['acc'])

In [55]:
model_simple_iitnet.fit([data_x],
                        data_y,
                        epochs=5)

Epoch 1/5
857/857 [==============================] - 31s 36ms/step - loss: 1.4091 - acc: 0.4096
Epoch 2/5
857/857 [==============================] - 21s 24ms/step - loss: 1.3365 - acc: 0.4294
Epoch 3/5
857/857 [==============================] - 22s 25ms/step - loss: 1.2790 - acc: 0.4364
Epoch 4/5
857/857 [==============================] - 22s 25ms/step - loss: 1.1888 - acc: 0.4889
Epoch 5/5
857/857 [==============================] - 22s 25ms/step - loss: 1.0805 - acc: 0.5554


In [56]:
model_simple_iitnet.evaluate(test_data_x, test_data_y)

795/795 [==============================] - 7s 8ms/step


[2.327806491072073, 0.42138364911079407]